In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plot
from sklearn import linear_model
from sklearn import metrics

CSV_PATH = "./transactions.csv"
df = pd.read_csv(CSV_PATH)

# Agency_0: 0
# Agency_1: 2,3,5
# Agency_2: 1,4

In [ ]:
# Task 1
agt = df['AgentId'].value_counts()

sum_agc = [agt[0],agt[2]+agt[3]+agt[5],agt[1]+agt[4]]

Agency = {'Agency Name':["Other","Your Estate","Lovely Housing"],\
        'Numbers':[sum_agc[0],sum_agc[1],sum_agc[2]]}

res_1 = pd.DataFrame(Agency)
res_1.sort_values("Numbers", ascending=False)

In [ ]:
# Task 2
df_2= df[['AgentId', 'Y house price of unit area']]
mean = []
for i in range(6):
    mean.append(df_2[df_2['AgentId'] == i]['Y house price of unit area'].mean())

# round mean price
for i in range(len(mean)):
    mean[i] = round(mean[i], 1)

Agent={'AgentId':[i for i in range(6)], \
        'Mean Price':mean}
res_2 = pd.DataFrame(Agent)
res_2.sort_values('Mean Price', ascending=False)

In [ ]:
# Task 3
df_3 = df[['AgentId','X1 transaction date','Y house price of unit area']]

# choose Agency
df_3_a0 = df_3[(df_3['AgentId']==0)]
df_3_a1 = df_3[(df_3['AgentId']==2) | (df_3['AgentId']==3) | (df_3['AgentId']==5)]
df_3_a2 = df_3[(df_3['AgentId']==1) | (df_3['AgentId']==4)]

# Year mean
mean_2012=round(df_3[df_3['X1 transaction date']<2013]['Y house price of unit area'].mean(), 1)
mean_2013=round(df_3[df_3['X1 transaction date']>=2013]['Y house price of unit area'].mean(), 1)

# Agency mean
mean_a0_2012=round(df_3_a0[df_3_a0['X1 transaction date']<2013]['Y house price of unit area'].mean(),1)
mean_a0_2013=round(df_3_a0[df_3_a0['X1 transaction date']>=2013]['Y house price of unit area'].mean(),1)

mean_a1_2012=round(df_3_a1[df_3_a1['X1 transaction date']<2013]['Y house price of unit area'].mean(),1)
mean_a1_2013=round(df_3_a1[df_3_a1['X1 transaction date']>=2013]['Y house price of unit area'].mean(),1)

mean_a2_2012=round(df_3_a2[df_3_a2['X1 transaction date']<2013]['Y house price of unit area'].mean(),1)
mean_a2_2013=round(df_3_a2[df_3_a2['X1 transaction date']>=2013]['Y house price of unit area'].mean(),1)

table_vals={"Year Mean Price": [mean_2012, mean_2013], \
            "Agency_0": [mean_a0_2012, mean_a0_2013], \
            "Agency_1": [mean_a1_2012, mean_a1_2013], \
            "Agency_2": [mean_a2_2012, mean_a2_2013]}

res_3 = pd.DataFrame(table_vals)
res_3.index = ['2012', '2013']
res_3

In [ ]:
# Task 4
res_3.plot(kind='bar', alpha=0.7)

In [ ]:
# Task 5
df_5 = df
corr_matrix = df_5.corr()
corr_matrix['Y house price of unit area'].sort_values()

## From this result, we choose X2,X3,X4,X5,X6 as X

In [ ]:
# split sets
from sklearn.model_selection import train_test_split

df_5 = df.drop(['AgentId', 'X1 transaction date'], axis=1)

data_x = df_5.drop('Y house price of unit area', axis=1)
data_y = df_5[['Y house price of unit area']]
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.2)

In [ ]:
# Task 6
# Linear Regeression
LR_reg = linear_model.LinearRegression()
LR_reg.fit(train_x, train_y)
print("LR:", LR_reg.coef_, LR_reg.intercept_)

# Test
preds = LR_reg.predict(test_x)
mse = metrics.mean_squared_error(test_y, preds)
mae = metrics.mean_absolute_error(test_y, preds)
mape = np.mean(np.abs((preds-test_y)/test_y))*100
print(mse, mae, mape)

In [ ]:
# Ridge Regression
# This model is better than Ridge
model_ridge =linear_model.Ridge(alpha = .5)
model_ridge.fit(train_x,train_y)
print("RR:", model_ridge.coef_, model_ridge.intercept_)

# Test
preds_ridge = model_ridge.predict(test_x)
mse_ridge = metrics.mean_squared_error(test_y, preds_ridge)
mae_ridge = metrics.mean_absolute_error(test_y, preds_ridge)
mape_ridge = np.mean(np.abs((preds_ridge-test_y)/test_y))*100
print(mse_ridge, mae_ridge, mape_ridge)

In [ ]:
# Task 7
plot.figure('model')
plot.plot(test_y, preds, '.')
# plot.scatter(test_y, preds)

plot.plot([test_y.min(), test_y.max()], [test_y.min(), test_y.max()], 'k--', lw =2)

plot.show()

如上图所示，蓝点的横坐标是实际值，纵坐标是预测值。

1. 如果蓝点位于黑色虚线的下方，代表实际值>预测值；
2. 如果蓝点位于黑色虚线的上方，代表实际值<预测值，
3. 如果蓝点位于黑色虚线上，代表实际值=预测值

Q1: Is your model overfitting? 

如上图所示，大部分点不在黑色虚线上，也就是训练集和预测集之间有一定差距，所以不是过拟合(overfitting)

Q2: Is your model underfitting?

使用线性回归模型，大部分点在黑色虚线附近，但是有个别点差距较大。但是由于数据集数量过少，所以很难说是否为欠拟合。